# Scenario Report

In [ ]:
# Copyright (c) 2021 *****************. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objects as go 
import numpy as np

#get the name of the current directory to specify the scenario name and identify the output directory
scenario_name = str(Path.cwd()).split('\\')[-1]
if scenario_name == 'inputs':
    data_dir = Path.cwd() / '../outputs/'
    inputs_dir = Path.cwd() / '../inputs/'
    scenario_name = 'N/A'
else:
    data_dir = Path.cwd() / f'../../outputs/{scenario_name}/'
    inputs_dir = Path.cwd() / f'../../inputs/{scenario_name}/'

#define formatting options/functions for outputs
pd.options.display.float_format = '{:,.2f}'.format
def format_currency(x): 
    try:
        formatted = '$ {:,.2f}'.format(x)
    except ValueError:
        formatted = x
    return formatted
def format_percent(x): 
    try:
        formatted = '{:,.2f}%'.format(x)
    except ValueError:
        formatted = x
    return formatted

#allow the notebook to display plots in html report
###################################################
plotly.offline.init_notebook_mode()

print(f'Scenario Name: {scenario_name}')

## Portfolio Renewable Percentage

In [ ]:
# Calculate Time-coincident renewable % and annual renewable %
##############################################################

# load load balance data    
load_balance = pd.read_csv(data_dir / 'load_balance.csv', parse_dates=True)

# get the list of generation column names that actually exist in the dataframe
generation_columns_list = ['ZoneTotalGeneratorDispatch','ZoneTotalVariableGeneration','ZoneTotalStorageDispatch']
generation_columns_available_list = [i for i in load_balance.columns if i in generation_columns_list]

# Calculate total generation from all sources
load_balance['Total_Generation'] = load_balance[generation_columns_available_list].sum(axis=1) 
load_balance['Total_Generation_No_Storage'] = load_balance[['ZoneTotalGeneratorDispatch','ZoneTotalVariableGeneration']].sum(axis=1)

# get the list of load column names that actually exist in the dataframe
load_columns_list = ['zone_demand_mw','ZoneTotalStorageCharge','ShiftDemand']
load_columns_available_list = [i for i in load_balance.columns if i in load_columns_list]

# Calculate total demand from all sources
load_balance['Total_Demand'] = load_balance[load_columns_available_list].sum(axis=1) 


# calculate net generation, treating storage as a supply-side resource
if 'ZoneTotalStorageCharge' in load_columns_available_list:
    load_balance['Net_Generation'] = load_balance['Total_Generation'] - load_balance['ZoneTotalStorageCharge']

    # calculate storage losses
    storage_losses = load_balance['ZoneTotalStorageCharge'].sum() - load_balance['ZoneTotalStorageDispatch'].sum()
else:
    load_balance['Net_Generation'] = load_balance['Total_Generation']

    # set storage losses = 0 if no storage in scenario
    storage_losses = 0

# calculate the time-coincident generation
# this calculation currently assumes that the batteries are only charging from contracted renewable generation
load_balance['Time_Coincident_Generation'] = load_balance[['Total_Generation','Total_Demand']].min(axis=1)
load_balance['Time_Coincident_Generation_No_Storage'] = load_balance[['Total_Generation_No_Storage','zone_demand_mw']].min(axis=1)

# TODO: Determine if SystemPower is ever used when not needed, which would affect the calculation
#calculate the time-coincident renewable percentage
try:
    tc_percent_renewable = (1 - load_balance['SystemPower'].sum() / load_balance['Total_Demand'].sum()) * 100
except KeyError:
    # if the SystemPower data is missing, then it is likely a 100% Time coincident scenario
    tc_percent_renewable = load_balance['Time_Coincident_Generation'].sum() / load_balance['Total_Demand'].sum() * 100

# calculate the time-coincident renewable % ignoring storage
tc_no_storage_percent_renewable = load_balance['Time_Coincident_Generation_No_Storage'].sum() / load_balance['zone_demand_mw'].sum() * 100

#tc_no_storage_percent_renewable = load_balance['Time_Coincident_Generation'].sum() / load_balance['zone_demand_mw'].sum() * 100
annual_percent_renewable = load_balance['Net_Generation'].sum() / load_balance['zone_demand_mw'].sum() * 100

print(f'Time-coincident renewable percentage (with storage dispatch): {np.round(tc_percent_renewable, decimals=2)}%')
print(f'Time-coincident renewable percentage (without storage dispatch): {np.round(tc_no_storage_percent_renewable, decimals=2)}%')
print(f'Annual volumetric renewable percentage: {annual_percent_renewable.round(decimals=2)}%')

## Generator Portfolio
The sunburst chart describes the built portfolio at various levels of detail, which shows how the outer rings relate to the inner rings
- Inner circle: contract status (contracted or additional project)
- Middle ring: technology type (e.g. solar, wind, ...)
- Outer ring: specific project name

For example, individual projects in the outer ring belong to a specific technology type in the middle ring, which can either be part of the existing/contracted portfolio, or the additional portfolio.


In [ ]:
# Prepare data for sunburst chart of portfolio
##############################################

#load capacity costs
portfolio = pd.read_csv(data_dir / 'gen_cap.csv', usecols=['generation_project','GenCapacity'])
portfolio = portfolio[portfolio['GenCapacity'] > 0]
portfolio = portfolio.rename(columns={'GenCapacity':'MW'})

# load data about predetermined generators
existing = pd.read_csv(inputs_dir / 'gen_build_predetermined.csv', usecols=['GENERATION_PROJECT']).rename(columns={'GENERATION_PROJECT':'generation_project'})


# add column indicating which generators are contracted or additional builds

existing['Status'] = 'Contracted'
portfolio = portfolio.merge(existing, how='left', on='generation_project').fillna('Additional')


#split the prefix of the generator name from the rest of the string
tech_column = [i.split('_')[0] for i in portfolio['generation_project']]
generator_name = [' '.join(i.split('_')[1:]) for i in portfolio['generation_project']]

#create a new tecnhology column based on the prefix and drop the generator name
portfolio['Technology'] = tech_column
portfolio['generation_project'] = generator_name

portfolio['MW'] = portfolio['MW'].round(1)

portfolio = portfolio.sort_values(by=['Status','Technology'])


portfolio_sunburst = px.sunburst(portfolio, path=['Status','Technology','generation_project'], values='MW', color='Technology', color_discrete_map={'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'Yellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'Green',
 '(?)':'Black'},
 width=1000, height=1000,
 title='Energy Portfolio by Project Name, Technology Type, and Contract Status (MW)')
portfolio_sunburst.update_traces(textinfo='label+value')
portfolio_sunburst.show()

In [ ]:
# Load Storage Costs
storage_costs = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=True, usecols=['generation_project','load_zone','timestamp','ChargeMW','DispatchMW','StorageDispatchPnodeCost'])

# merge system power cost data
storage_costs = storage_costs.merge(system_power_cost, how='left', on=['timestamp','load_zone'])

# split the hybrid generators into a separate df
hybrid_costs = storage_costs.copy()[storage_costs['generation_project'].str.contains('HYBRID')]
storage_costs = storage_costs[~storage_costs['generation_project'].str.contains('HYBRID')]


## STORAGE
# calculate delivery cost 
storage_costs['Delivery Cost'] = storage_costs['DispatchMW'] * storage_costs['system_power_cost']
# calculate Pnode revenue from discharge
storage_costs['Pnode Revenue'] = - storage_costs['StorageDispatchPnodeCost']
storage_costs.loc[storage_costs['Pnode Revenue'] < 0,'Pnode Revenue'] = 0  
# calculate congestion cost
storage_costs['Congestion Cost'] = storage_costs['Delivery Cost'] - storage_costs['Pnode Revenue']
# calculate annual sums and drop data with zero values
storage_costs = storage_costs.groupby('generation_project').sum().reset_index()
storage_costs = storage_costs[storage_costs['DispatchMW'] > 0]
#merge in the contract costs
storage_contract = pd.read_csv(data_dir / 'gen_cap.csv', usecols=['generation_project','Annual_PPA_Capacity_Cost'])
storage_costs = storage_costs.merge(storage_contract, how='left', on='generation_project').fillna(0)
# clean up columns
storage_costs = storage_costs.rename(columns={'StorageDispatchPnodeCost':'Energy Arbitrage','Annual_PPA_Capacity_Cost':'Contract Cost','DispatchMW':'Generation MWh'})
storage_costs = storage_costs[['generation_project','Generation MWh','Contract Cost','Energy Arbitrage','Pnode Revenue', 'Delivery Cost', 'Congestion Cost']]

## HYBRID
# calculate net delivery cost 
hybrid_costs['Delivery Cost'] = (hybrid_costs['DispatchMW'] - hybrid_costs['ChargeMW']) * hybrid_costs['system_power_cost']
# calculate net Pnode revenue
hybrid_costs['Pnode Revenue'] = - hybrid_costs['StorageDispatchPnodeCost']
# calculate net congestion cost
hybrid_costs['Congestion Cost'] = hybrid_costs['Delivery Cost'] - hybrid_costs['Pnode Revenue']
# calculate annual sums and drop data with zero values
hybrid_costs = hybrid_costs.groupby('generation_project').sum().reset_index()
hybrid_costs = hybrid_costs[hybrid_costs['DispatchMW'] > 0]
#merge in the contract costs
hybrid_costs = hybrid_costs.merge(storage_contract, how='left', on='generation_project').fillna(0)
# calculate net generation
hybrid_costs['Generation MWh'] = hybrid_costs['DispatchMW'] - hybrid_costs['ChargeMW']
# clean up columns
hybrid_costs = hybrid_costs.rename(columns={'Annual_PPA_Capacity_Cost':'Contract Cost'})
hybrid_costs = hybrid_costs[['generation_project','Generation MWh','Contract Cost','Pnode Revenue', 'Delivery Cost', 'Congestion Cost']]
#rename storage hybrid to paired generator name
hybrid_pair = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','storage_hybrid_generation_project'])
hybrid_pair = hybrid_pair[hybrid_pair['storage_hybrid_generation_project'] != "."]
hybrid_pair = dict(zip(hybrid_pair.GENERATION_PROJECT, hybrid_pair.storage_hybrid_generation_project))
hybrid_costs['generation_project'] = hybrid_costs['generation_project'].replace(hybrid_pair)

# load generator data
generator_costs = pd.read_csv(data_dir / 'nodal_costs_by_gen.csv')
# clean up the columns
generator_costs = generator_costs.rename(columns={'DispatchGen_MW':'Generation MWh', 'Generator Pnode Revenue':'Pnode Revenue'})

# load generator load zone data and merge into generator costs data
generator_load_zones = pd.read_csv(inputs_dir / 'generation_projects_info.csv', usecols=['GENERATION_PROJECT','gen_load_zone']).rename(columns={'GENERATION_PROJECT':'generation_project', 'gen_load_zone':'load_zone'})
generator_costs = generator_costs.merge(generator_load_zones, how='left', on='generation_project')

# load system power cost and merge into generator cost data
system_power_cost = pd.read_csv(inputs_dir / 'system_power_cost.csv', usecols=['load_zone','system_power_cost'])
system_power_cost['timestamp'] = generator_costs['timestamp'].iloc[:8760]
generator_costs = generator_costs.merge(system_power_cost, how='left', on=['timestamp','load_zone'])

# calculate delivery costs and congestion cost
generator_costs['Delivery Cost'] = generator_costs['Generation MWh'] * generator_costs['system_power_cost']
generator_costs['Congestion Cost'] = generator_costs['Delivery Cost'] - generator_costs['Pnode Revenue']

# clean up columns
generator_costs = generator_costs[['generation_project','Generation MWh','Contract Cost','Pnode Revenue', 'Delivery Cost', 'Congestion Cost']]

# sum the costs for the entire year
# calculate annual sums and drop data with zero values
generator_costs = generator_costs.groupby('generation_project').sum().reset_index()
generator_costs = generator_costs[generator_costs['Generation MWh'] > 0]

# append the hybrid and storage data and group the hybrid components together
generator_costs = generator_costs.append(hybrid_costs).fillna(0)
generator_costs = generator_costs.append(storage_costs).fillna(0)
generator_costs = generator_costs.groupby('generation_project').sum().reset_index()

# calculate per MWh costs
generator_costs['Contract Cost per MWh'] = generator_costs['Contract Cost'] / generator_costs['Generation MWh']
generator_costs['Congestion Cost per MWh'] = generator_costs['Congestion Cost'] / generator_costs['Generation MWh']
generator_costs['Energy Arbitrage per MWh'] = generator_costs['Energy Arbitrage'] / generator_costs['Generation MWh']
generator_costs['Total Cost per MWh'] = generator_costs['Contract Cost per MWh'] + generator_costs['Congestion Cost per MWh'] + generator_costs['Energy Arbitrage per MWh']

generator_costs['Technology'] = [i.split('_')[0] for i in generator_costs['generation_project']]

generator_costs = generator_costs.sort_values(by='Total Cost per MWh', ascending=True)

generator_costs = generator_costs.round(decimals=2)

# Prepare Data for graph
########################
generator_costs_melted = generator_costs[['generation_project', 'Contract Cost per MWh','Congestion Cost per MWh','Energy Arbitrage per MWh']] \
                            .rename(columns={'Contract Cost per MWh':'Contract Cost','Congestion Cost per MWh':'Congestion Cost','Energy Arbitrage per MWh':'Energy Arbitrage'}) \
                            .melt(id_vars='generation_project', var_name='Cost', value_name='$/MWh')

generator_cost_fig = px.bar(generator_costs_melted, title='Average Generator Cost per MWh Generated', x='generation_project', y='$/MWh', color='Cost', color_discrete_map={'Contract Cost': 'Green','Congestion Cost': 'Orange', 'Energy Arbitrage':'Blue'})


generator_cost_fig.add_scatter(x=generator_costs.generation_project, y=generator_costs['Total Cost per MWh'], mode='markers+text', text=generator_costs['Total Cost per MWh'], textposition='top center', line=dict(color='black', width=1), name='Total Cost')

generator_cost_fig.show()


## Delivered Energy Mix
This is similar to a power content label, showing the percentage of energy from each source used to serve load.

In [ ]:
total_demand = load_balance['zone_demand_mw'].sum()
try:
    system_power_consumed = load_balance['SystemPower'].sum()
except KeyError:
    system_power_consumed = 0
generated_power_consumed = total_demand - system_power_consumed

# Calculate generated energy mix
generation_mix = pd.read_csv(data_dir / 'dispatch_annual_summary.csv', usecols=['gen_energy_source','Energy_GWh_typical_yr'])
# calculate the % generation from each technology
generation_mix['Energy_GWh_typical_yr'] = generation_mix['Energy_GWh_typical_yr'] / generation_mix['Energy_GWh_typical_yr'].sum()
# calculate the total MWh of consumed power from each source
generation_mix['Energy_GWh_typical_yr'] = generation_mix['Energy_GWh_typical_yr'] * generated_power_consumed
# rename the columns
generation_mix = generation_mix.rename(columns={'gen_energy_source':'Source','Energy_GWh_typical_yr':'Delivered MWh'})
# add a row for system power
generation_mix = generation_mix.append({'Source':'System_Power','Delivered MWh':system_power_consumed}, ignore_index=True)


generation_mix['Delivered MWh'] = generation_mix['Delivered MWh'].round(decimals=0)

# only keep non-zero data
generation_mix = generation_mix[generation_mix['Delivered MWh'] > 0]

# rename water to Hydro
generation_mix = generation_mix.replace('Water','Hydro')

delivered_energy_pie = px.pie(generation_mix, values='Delivered MWh', names='Source', title='Delivered Energy Mix (MWh)', color='Source', color_discrete_map={'Hydro':'Purple',
 'Onshore_Wind':'Blue',
 'Offshore_Wind':'Navy',
 'Solar':'Yellow',
 'Geothermal':'Sienna',
 'System_Power':'Red'},
width=600, height=600
 )
delivered_energy_pie.update_traces(textinfo='percent+label+value')
delivered_energy_pie.show()


## Delivered Electricity Cost Summary

In [ ]:
costs_itemized_df = pd.read_csv(data_dir / 'costs_itemized.csv')
try:
    nodal_costs_df = pd.read_csv(data_dir / 'nodal_costs.csv')
    nodal_costs_exist = True
except FileNotFoundError:
    nodal_costs_exist = False

# Energy Contract Cost
energy_contract_cost = costs_itemized_df[costs_itemized_df['Component'] == 'GenPPACostInTP']['AnnualCost_Real'].to_numpy()[0]

# Capacity Contract Cost
capacity_contract_cost = costs_itemized_df[costs_itemized_df['Component'] == 'TotalGenCapacityCost']['AnnualCost_Real'].to_numpy()[0]

# Delivered Energy Cost
if nodal_costs_exist:
    delivered_energy_cost = nodal_costs_df['DLAP Cost'].sum()

# Generator Pnode Revenue
if nodal_costs_exist:
    generator_pnode_revenue = - nodal_costs_df['Generator Pnode Revenue'].sum()

# Storage Energy Arbitrage
try:
    storage_energy_arbitrage_cost = costs_itemized_df[costs_itemized_df['Component'] == 'StorageNodalEnergyCostInTP']['AnnualCost_Real'].to_numpy()[0]
except IndexError:
    storage_energy_arbitrage_cost = 0

# Resource Adequacy Costs
try:
    ra_open_position_cost = costs_itemized_df[costs_itemized_df['Component'] == 'TotalRAOpenPositionCost']['AnnualCost_Real'].to_numpy()[0]
except IndexError:
    ra_open_position_cost = 0

try:
    # Build the cost table
    cost = {
            'Cost Component': ['Energy Contract Cost','Storage Capacity Cost','Delivered Energy Cost','Generator Pnode Revenue','Storage Energy Arbitrage','Resource Adequacy Open Position Cost'],
            'Annual Real Cost': [energy_contract_cost, capacity_contract_cost, delivered_energy_cost, generator_pnode_revenue, storage_energy_arbitrage_cost, ra_open_position_cost]
        }

    cost_summary = pd.DataFrame(cost)
    cost_summary = cost_summary.round(decimals=2)
    #drop any rows with zero values
    cost_summary = cost_summary[cost_summary['Annual Real Cost'] != 0]

    #add a total row
    cost_summary = cost_summary.append({'Cost Component':'TOTAL','Annual Real Cost':cost_summary['Annual Real Cost'].sum()}, ignore_index=True)

    # add a column for delivered cost
    cost_summary['Delivered Cost per MWh'] = (cost_summary['Annual Real Cost'] / total_demand).round(decimals=2)

    # export calculated costs to csv
    unformatted_cost = cost_summary.copy()
    unformatted_cost.to_csv(data_dir / 'total_cost_of_energy.csv')

    cost_summary['Delivered Cost per MWh'] = cost_summary['Delivered Cost per MWh'].apply(format_currency)
    cost_summary['Annual Real Cost'] = cost_summary['Annual Real Cost'].apply(format_currency)
    cost_summary = cost_summary.set_index('Cost Component')

    display(cost_summary)
except NameError:
    pass


## Excess Generation Summary

In [ ]:
try:
    total_generation_mwh = load_balance['Total_Generation_No_Storage'].sum()
    excess_tc_generation_mwh = (load_balance['Total_Generation_No_Storage'] - load_balance['Time_Coincident_Generation']).sum()
    excess_volumetric_generation_mwh = load_balance['Total_Generation_No_Storage'].sum() - load_balance['Total_Demand'].sum() 


    generation_avg_contract_cost_per_mwh = energy_contract_cost / total_generation_mwh
    generation_avg_pnode_revenue_per_mwh = generator_pnode_revenue / total_generation_mwh

    if excess_tc_generation_mwh > 0:
        print(f'Excess generation (RECs) by annual volume: {int(excess_volumetric_generation_mwh)} MWh')
        print(f'Excess time-coincident generation (hourly RECs): {int(excess_tc_generation_mwh)} MWh\n')
        
        print(f'Average Contract Cost of Generation: {format_currency(generation_avg_contract_cost_per_mwh)} / MWh generated')
        print(f'Average Pnode Cost of Generation: {format_currency(generation_avg_pnode_revenue_per_mwh)} / MWh generated')
        print(f'Average Total Cost of Generation: {format_currency(generation_avg_contract_cost_per_mwh+generation_avg_pnode_revenue_per_mwh)} / MWh generated\n')
        
        print(f'Total Contract Cost of Excess Time-coincident Generation: {format_currency(generation_avg_contract_cost_per_mwh*excess_tc_generation_mwh)}')
        print(f'Total Pnode Cost of Excess Time-coincident Generation: {format_currency(generation_avg_pnode_revenue_per_mwh*excess_tc_generation_mwh)}')
        print(f'Total Cost of Excess Time-coincident Generation: {format_currency((generation_avg_contract_cost_per_mwh+generation_avg_pnode_revenue_per_mwh)*excess_tc_generation_mwh)}')
    else:
        print('No excess generation')
except NameError:
    pass

## Resource Adequacy Open Position

In [ ]:
try:
    ra_open = pd.read_csv(data_dir / 'RA_open_position.csv')
    
    #where RA Position is negative, it indicates an open position
    ra_open['Color'] = np.where(ra_open["RA_Position_MW"]<0, 'Open Position', 'Excess RA')

    #create a plot of monthly RA position
    monthly_ra_open_fig = px.bar(ra_open, x='Month', y='RA_Position_MW', facet_col='RA_Requirement', color='Color', color_discrete_map={'Excess RA':'green', 'Open Position':'red'}, title='Monthly RA position by requirement')
    monthly_ra_open_fig.for_each_annotation(lambda a: a.update(text=a.text.replace("RA_Requirement=", "")))

    sellable_flex = ra_open.copy()
    sellable_flex = sellable_flex[(sellable_flex['RA_Requirement'] == 'flexible_RA') & (sellable_flex['RA_Position_MW'] > 0)]

    #flex RA must be paired with regular RA to sell, so we need to limit it
    def calculate_sellable_flex(row, ra_open):
        # get the month number
        month = row['Month']
        #get the amount of excess system RA
        system_ra_open_MW = ra_open.loc[(ra_open['RA_Requirement'] == 'system_RA') & (ra_open['Month'] == month), 'Excess_RA_MW'].item()
        # find the minimum of the excess system RA and excess flex RA
        sellable = min(row['RA_Position_MW'], system_ra_open_MW)
        return sellable   
    sellable_flex.loc[:,'Sellable_Flex_MW'] = sellable_flex.apply(lambda row: calculate_sellable_flex(row, ra_open), axis=1)

    #re-calculate the sellable position of flex RA
    sellable_flex['Excess_RA_Value'] = sellable_flex['RA_Value'] * sellable_flex['Sellable_Flex_MW']

    #calculate how much system could be sold for subtracting the local
    sellable_system = ra_open[(ra_open['RA_Requirement'] == 'system_RA')]

    local_RA = ra_open[(ra_open['RA_Requirement'] != 'system_RA') & (ra_open['RA_Requirement'] != 'flexible_RA')][['Month','Excess_RA_MW']]
    local_RA = local_RA.groupby('Month').sum().reset_index().rename(columns={'Excess_RA_MW':'Local_RA_MW'})

    #merge local RA data into system data
    sellable_system = sellable_system.merge(local_RA, how='left', on='Month').fillna(0)
    sellable_system['Sellable_System_MW'] = sellable_system['Excess_RA_MW'] - sellable_system['Local_RA_MW']

    sellable_system['Excess_RA_Value'] = sellable_system['RA_Value'] * sellable_system['Sellable_System_MW']

    #calculate total RA costs and value
    total_RA_open_cost = ra_open['Open_Position_Cost'].sum()
    excess_flex_RA_value = sellable_flex['Excess_RA_Value'].sum()
    excess_local_RA_value = ra_open.loc[(ra_open['RA_Requirement'] != 'system_RA') & (ra_open['RA_Requirement'] != 'flexible_RA'), 'Excess_RA_Value'].sum()
    excess_system_RA_value = sellable_system['Excess_RA_Value'].sum()

    #calculate the best-case RA cost if all RA can be sold at full market value
    #ra_cost_best_case = (total_RA_open_cost - (excess_flex_RA_value + excess_local_RA_value + excess_system_RA_value)) / system_demand_mwh

    #create cost summary of RA costs
    ################################

    ra_costs = ra_open.copy()
    #groupby RA requirement
    ra_costs = ra_costs.groupby('RA_Requirement').mean().drop(columns=['Period','Month'])
    #rename columns
    ra_costs = ra_costs.rename(columns={'Open_Position_MW': 'Monthly_Avg_Open_Position_MW', 'Open_Position_Cost': 'Monthly_Avg_Open_Position_Cost','RA_Requirement_Need_MW':'Monthly_Avg_RA_Need_MW','Available_RA_Capacity_MW':'Monthly_Avg_Available_RA_MW'})

    
    #calculate the annual total
    ra_costs['Annual_Open_Position_Cost'] = ra_costs['Monthly_Avg_Open_Position_Cost'] * 12
    ra_costs['$/MWh delivered'] = ra_costs['Annual_Open_Position_Cost'] / total_demand
    #ra_costs['Weighted Cost'] = ra_costs['RA_Cost'] / 1000
    #ra_costs['Unit'] = '$/kW-mo'
    ra_costs = ra_costs[['$/MWh delivered']]
    #ra_costs = ra_costs.drop(columns=['Monthly_Avg_Open_Position_Cost','Monthly_Avg_RA_Need_MW','Monthly_Avg_Available_RA_MW','Monthly_Avg_Open_Position_MW','Annual_Open_Position_Cost','RA_Position_MW', 'Excess_RA_MW','RA_Cost','RA_Value','Excess_RA_Value'])

    #merge RA cost data into cost_summary
    ra_costs = ra_costs.reset_index().rename(columns={'RA_Requirement':'Cost Component'})

    print(f'Resale value of Excess RA: {format_currency(excess_flex_RA_value + excess_local_RA_value + excess_system_RA_value)}')
except FileNotFoundError:
    pass

In [ ]:
try:
    monthly_ra_open_fig.show()
except:
    print('Resource Adequacy was not considered in this scenario')

# Supply and Demand Balance

In [ ]:
# Set up data for dispatch timeseries graph
###########################################

dispatch = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','generation_project','DispatchGen_MW'], parse_dates=['timestamp'], infer_datetime_format=True)

"""

#create a dictionary that matches hybrid storage to generator
hybrid_pair = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','storage_hybrid_generation_project'])
hybrid_pair = hybrid_pair[hybrid_pair['storage_hybrid_generation_project'] != "."]
hybrid_pair = dict(zip(hybrid_pair.GENERATION_PROJECT, hybrid_pair.storage_hybrid_generation_project))
"""

try:
    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)
    """
    #merge charging data for hybrid storage projects into the dataframe
    hybrid_dispatch = storage.copy()[['generation_project','timestamp','ChargeMW','DispatchMW']]
    hybrid_dispatch.loc[:,'generation_project'] = hybrid_dispatch['generation_project'].replace(hybrid_pair)
    dispatch = dispatch.merge(hybrid_dispatch, how='left', on=['generation_project','timestamp']).fillna(0)

    # subtract out charging energy from generation from hybrid projects to get true generation
    dispatch.loc[:,'DispatchGen_MW'] = dispatch['DispatchGen_MW'] - dispatch['ChargeMW'] + dispatch['DispatchMW']
    #drop the hybrid columns
    dispatch = dispatch.drop(columns=['ChargeMW','DispatchMW'])
    """
    # rename the column
    dispatch = dispatch.rename(columns={'DispatchGen_MW':'DispatchMW'})


    #append storage 
    storage_dispatch = storage[['generation_project','timestamp','DispatchMW']]
    #storage_dispatch = storage_dispatch[~storage_dispatch['generation_project'].str.contains('HYBRID')]
    dispatch = dispatch.append(storage_dispatch)

    # Setup storage charging timeseries data
    ########################################
    storage_charge = storage[['generation_project','timestamp','ChargeMW']]
    #storage_charge = storage_charge[~storage_charge['generation_project'].str.contains('HYBRID')]
    storage_charge = storage_charge.groupby('timestamp').sum().reset_index()

except FileNotFoundError:
    #calculate total generation
    dispatch = dispatch.rename(columns={'DispatchGen_MW':'DispatchMW'})

#add system power data if available
try:
    system_power_use = pd.read_csv(data_dir / 'system_power.csv', usecols=['timestamp','System_Power_MW'], parse_dates=['timestamp'], infer_datetime_format=True)
    system_power_use.loc[:,'generation_project'] = 'SYSTEM_POWER'
    system_power_use = system_power_use.rename(columns={'System_Power_MW':'DispatchMW'})
    dispatch = dispatch.append(system_power_use)

except FileNotFoundError:
    pass

#add technology column
dispatch['Technology'] = [i.split('_')[0] for i in dispatch['generation_project']]

# replace the technology name of the RE portion of a hybrid with just the name of the technology
for tech in list(dispatch['Technology'].unique()):
    if 'HYBRID' in tech and 'STORAGE' not in tech:
        dispatch.loc[dispatch['Technology'] == tech, 'Technology'] = tech.replace('HYBRID','')

# group the data by technology type
dispatch_by_tech = dispatch.groupby(['Technology','timestamp']).sum().reset_index()
#only keep observations greater than 0
dispatch_by_tech = dispatch_by_tech[dispatch_by_tech['DispatchMW'] > 0]
# Setup load timeseries data
############################

load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], parse_dates=['timestamp'], infer_datetime_format=True)

# Create Figure
###############

color_map = {'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'GreenYellow',
 'SYSTEM':'Red'}

dispatch_fig = px.area(dispatch_by_tech, x='timestamp', y='DispatchMW', color='Technology', color_discrete_map=color_map, labels={'DispatchMW':'MW','timestamp':'Datetime','Technology':'Key'})
dispatch_fig.update_traces(line={'width':0})
#dispatch_fig.update_traces(hoveron="points+fills")
dispatch_fig.layout.template = 'plotly_white'
#hoveron = 'points+fills'

try:
    # add charging line
    dispatch_fig.add_scatter(x=storage_charge.timestamp, y=(storage_charge.ChargeMW + load.zone_demand_mw), line=dict(color='green', width=4), name='Load + Storage Charge')
except NameError:
    pass

# add load line
dispatch_fig.add_scatter(x=load.timestamp, y=load.zone_demand_mw, line=dict(color='black', width=4), name='Demand')

dispatch_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))

dispatch_fig.show()

In [ ]:
# Nodal Energy Cost data
nodal_prices = pd.read_csv(inputs_dir / 'nodal_prices.csv')
timestamps = pd.read_csv(inputs_dir / 'timepoints.csv', parse_dates=['timestamp'], usecols=['timepoint_id','timestamp'])
# merge the timestamp data
nodal_prices = nodal_prices.merge(timestamps, how='left', left_on='timepoint', right_on='timepoint_id')

nodal_fig = px.line(nodal_prices, x='timestamp', y='nodal_price', color='pricing_node', labels={'nodal_price':'$/MWh','timestamp':'Datetime','pricing_node':'Node'}, title='Nodal Prices', template='plotly_white')
nodal_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))
nodal_fig.show()

In [ ]:
# Battery State of Charge data
##############################

try:

    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)
    
    soc = storage.pivot(index='timestamp', columns='generation_project', values='StateOfCharge')
    #remove columns where all values are 0
    soc = soc.loc[:, (soc != 0).any(axis=0)]
    
    #load storage capacity
    storage_energy_capacity = pd.read_csv(data_dir / 'storage_builds.csv', usecols=['generation_project','OnlineEnergyCapacityMWh'], index_col='generation_project')

    #create another dictionary of storage energy capacity summed by storage type
    storage_energy_capacity = storage_energy_capacity.reset_index()
    storage_energy_capacity['generation_project'] = [i.split('_')[0] for i in storage_energy_capacity['generation_project']]
    storage_energy_capacity = storage_energy_capacity.groupby('generation_project').sum()
    grouped_storage_energy_capacity_dict = storage_energy_capacity.to_dict()['OnlineEnergyCapacityMWh']


    #sum by storage type
    soc.columns = [i.split('_')[0] for i in soc.columns]
    
    soc = soc.groupby(soc.columns, axis=1).sum()
    
    #divide by the total capacity to get state of charge
    soc = soc.div(soc.assign(**grouped_storage_energy_capacity_dict))
    
    #soc.index = pd.to_datetime(soc.index)
    
    soc.head(5)

    soc_fig = px.line(soc, x=soc.index, y=list(soc.columns), color_discrete_map={'STORAGE':'green','STORAGEHYBRID':'yellowgreen'}, labels={'timestamp':'Datetime','value':'%'}, title='Storage State of Charge')

    soc_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))

    soc_fig.show()
    
except FileNotFoundError:
    pass

In [ ]:
mh_dispatch = dispatch.copy()
mh_dispatch = mh_dispatch.set_index('timestamp')

#groupby month and hour
mh_dispatch = mh_dispatch.groupby(['generation_project', mh_dispatch.index.month, mh_dispatch.index.hour], axis=0).mean()
mh_dispatch.index = mh_dispatch.index.rename(['Project','Month','Hour'])
mh_dispatch = mh_dispatch.reset_index()

#add a technology column
mh_dispatch['Technology'] = mh_dispatch['Project'].str.split('_', expand=True)[0]

mh_dispatch = mh_dispatch[mh_dispatch['DispatchMW'] > 0]


#load data
mh_load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], index_col='timestamp', parse_dates=True, infer_datetime_format=True).rename(columns={'zone_demand_mw':'DEMAND'})
mh_load = mh_load.groupby([mh_load.index.month, mh_load.index.hour], axis=0).mean()
mh_load.index = mh_load.index.rename(['Month','Hour'])
mh_load = mh_load.reset_index()

try:
    mh_charge = storage_charge.copy().set_index('timestamp')
    mh_charge = mh_charge.groupby([mh_charge.index.month, mh_charge.index.hour], axis=0).mean()
    mh_charge.index = mh_charge.index.rename(['Month','Hour'])
    mh_charge = mh_charge.reset_index()
    #merge load data
    mh_charge = mh_charge.merge(mh_load, how='left', on=['Month','Hour'])
    mh_charge['ChargeMW'] = mh_charge['ChargeMW'] + mh_charge['DEMAND']
except NameError:
    pass


# Generate the Figure
#####################

color_map = {'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'GreenYellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'GreenYellow',
 'SYSTEM':'Red'}

mh_fig = px.area(mh_dispatch, x='Hour', y='DispatchMW', facet_col='Month', color='Technology', line_group='Project', color_discrete_map=color_map, facet_col_wrap=6, width=1000, height=600, title='Month-Hour Average Generation Profiles')
mh_fig.layout.template = 'plotly_white'
mh_fig.update_traces(line={'dash':'dot'})

try:
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 1, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 1, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=1, name='STORAGE_Charge', showlegend=True)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 2, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 2, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=2, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 3, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 3, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=3, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 4, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 4, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=4, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 5, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 5, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=5, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 6, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 6, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=6, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 7, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 7, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=1, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 8, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 8, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=2, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 9, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 9, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=3, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 10, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 10, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=4, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 11, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 11, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=5, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 12, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 12, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=6, name='STORAGE_Charge', showlegend=False)
except (NameError, KeyError) as e:
    pass

mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 1, 'Hour'], y=mh_load.loc[mh_load['Month'] == 1, 'DEMAND'], line=dict(color='black', width=4), row=2, col=1, name='Demand', showlegend=True)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 2, 'Hour'], y=mh_load.loc[mh_load['Month'] == 2, 'DEMAND'], line=dict(color='black', width=4), row=2, col=2, name='Demand',showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 3, 'Hour'], y=mh_load.loc[mh_load['Month'] == 3, 'DEMAND'], line=dict(color='black', width=4), row=2, col=3, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 4, 'Hour'], y=mh_load.loc[mh_load['Month'] == 4, 'DEMAND'], line=dict(color='black', width=4), row=2, col=4, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 5, 'Hour'], y=mh_load.loc[mh_load['Month'] == 5, 'DEMAND'], line=dict(color='black', width=4), row=2, col=5, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 6, 'Hour'], y=mh_load.loc[mh_load['Month'] == 6, 'DEMAND'], line=dict(color='black', width=4), row=2, col=6, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 7, 'Hour'], y=mh_load.loc[mh_load['Month'] == 7, 'DEMAND'], line=dict(color='black', width=4), row=1, col=1, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 8, 'Hour'], y=mh_load.loc[mh_load['Month'] == 8, 'DEMAND'], line=dict(color='black', width=4), row=1, col=2, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 9, 'Hour'], y=mh_load.loc[mh_load['Month'] == 9, 'DEMAND'], line=dict(color='black', width=4), row=1, col=3, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 10, 'Hour'], y=mh_load.loc[mh_load['Month'] == 10, 'DEMAND'], line=dict(color='black', width=4), row=1, col=4, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 11, 'Hour'], y=mh_load.loc[mh_load['Month'] == 11, 'DEMAND'], line=dict(color='black', width=4), row=1, col=5, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 12, 'Hour'], y=mh_load.loc[mh_load['Month'] == 12, 'DEMAND'], line=dict(color='black', width=4), row=1, col=6, name='Demand', showlegend=False)

month_names = ['July', 'August', 'September', 'October', 'November', 'December','January', 'February', 'March', 'April', 'May', 'June']
for i, a in enumerate(mh_fig.layout.annotations):
    a.text = month_names[i]


mh_fig.update_xaxes(dtick=3)
mh_fig.show()

In [ ]:
# get month-hour shape of open energy position
##############################################

dispatch_gen = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','DispatchGen_MW'], parse_dates=['timestamp'], infer_datetime_format=True)

#sum by timestamp
dispatch_gen = dispatch_gen.groupby('timestamp').sum().reset_index()

#sum to get total generation
dispatch_gen['Generation_MW'] = dispatch_gen['DispatchGen_MW']

#merge generation_load_mismatch data
generation_load_mismatch = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], parse_dates=['timestamp'], infer_datetime_format=True)

generation_load_mismatch = generation_load_mismatch.merge(dispatch_gen[['timestamp','Generation_MW']], how='left', on='timestamp')

#subtract generation from generation_load_mismatch to get open position and replace negative values with zero
generation_load_mismatch['Overgeneration_MW'] = generation_load_mismatch['Generation_MW'] - generation_load_mismatch['zone_demand_mw']
#generation_load_mismatch.loc[generation_load_mismatch['generation_load_mismatch_MW'] < 0, 'generation_load_mismatch_MW'] = 0

#generation_load_mismatch['timestamp'] = pd.to_datetime(generation_load_mismatch['timestamp'])

generation_load_mismatch = generation_load_mismatch.set_index('timestamp')

generation_load_mismatch_mh = generation_load_mismatch.groupby([generation_load_mismatch.index.month, generation_load_mismatch.index.hour], axis=0).mean()
generation_load_mismatch_mh.index = generation_load_mismatch_mh.index.rename(['Month','Hour'])
generation_load_mismatch_mh = generation_load_mismatch_mh.reset_index()

mh_mismatch_fig = px.area(generation_load_mismatch_mh, x='Hour', y='Overgeneration_MW', facet_col='Month', facet_col_wrap=6, width=1000, height=600, title='Shape of over- and under-generation, excluding energy storage')
mh_mismatch_fig.update_xaxes(dtick=3)
month_names = ['July', 'August', 'September', 'October', 'November', 'December','January', 'February', 'March', 'April', 'May', 'June']
for i, a in enumerate(mh_mismatch_fig.layout.annotations):
    a.text = month_names[i]

mh_mismatch_fig.show()



## Battery Cycles

In [ ]:
try:
    cycles = pd.read_csv(data_dir / 'storage_cycle_count.csv', usecols=['generation_project','storage_max_annual_cycles','Battery_Cycle_Count'])
    cycles = cycles.round(decimals=2)
    cycles = cycles[cycles['Battery_Cycle_Count'] > 0]
    storage_energy_capacity = pd.read_csv(data_dir / 'storage_builds.csv', usecols=['generation_project','OnlineEnergyCapacityMWh'])
    cycles = cycles.merge(storage_energy_capacity, how='left', on='generation_project')
    cycles['Battery_Cycle_Count'] = cycles['Battery_Cycle_Count'] / cycles['OnlineEnergyCapacityMWh']
    cycles = cycles.drop(columns=['OnlineEnergyCapacityMWh']).set_index('generation_project').round(decimals=0)
    cycles['Battery_Cycle_Count'] = cycles['Battery_Cycle_Count'].astype(int)
except FileNotFoundError:
    cycles = "No batteries in model"
cycles

# Generator Assumptions

In [ ]:
pd.set_option('display.max_rows',100)
gen_assumptions = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','gen_tech','gen_energy_source','gen_reliability_area','ppa_energy_cost','gen_capacity_limit_mw'])
gen_assumptions = gen_assumptions[gen_assumptions['gen_energy_source'] != 'Electricity']
gen_assumptions = gen_assumptions.sort_values(by='GENERATION_PROJECT')
gen_assumptions = gen_assumptions.set_index('GENERATION_PROJECT')
gen_assumptions

# Storage Assumptions

In [ ]:
storage_assumptions = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','gen_tech','gen_energy_source','gen_reliability_area','ppa_capacity_cost','gen_capacity_limit_mw', 'storage_roundtrip_efficiency','storage_charge_to_discharge_ratio','storage_energy_to_power_ratio','storage_leakage_loss','storage_hybrid_generation_project','storage_hybrid_capacity_ratio'])
#change capacity cost to $/kw-mo
storage_assumptions['ppa_capacity_cost'] = storage_assumptions['ppa_capacity_cost'] / 12000

storage_assumptions = storage_assumptions.rename(columns={
    'storage_roundtrip_efficiency':'RTE',
    'storage_charge_to_discharge_ratio':'charge/discharge_ratio',
    'storage_energy_to_power_ratio':'storage_hours',
    'storage_leakage_loss':'soc_leakage_loss',
    'storage_hybrid_generation_project':'paired_hybrid_gen',
    'storage_hybrid_capacity_ratio':'hybrid_capacity_ratio'
})

storage_assumptions = storage_assumptions[storage_assumptions['gen_energy_source'] == 'Electricity']
storage_assumptions = storage_assumptions.sort_values(by='GENERATION_PROJECT')
storage_assumptions = storage_assumptions.set_index('GENERATION_PROJECT')
storage_assumptions = storage_assumptions[['gen_tech','gen_reliability_area','ppa_capacity_cost','gen_capacity_limit_mw', 'RTE','storage_hours','charge/discharge_ratio','soc_leakage_loss','paired_hybrid_gen','hybrid_capacity_ratio']]
storage_assumptions

In [ ]:
summary = pd.DataFrame(columns=['Scenario Name'], data=['test'])

summary['Scenario Name'] = scenario_name

#Goal Data
summary['Time-coincident Delivered %'] = tc_percent_renewable
summary['Time-coincident Generation %'] = tc_no_storage_percent_renewable
summary['Annual Volumetric Renewable %'] = annual_percent_renewable

unformatted_cost = unformatted_cost.rename(columns={'Annual Real Cost': ' (Annual)',	'Delivered Cost per MWh': ' (per MWh)'}).melt(id_vars=['Cost Component'], var_name='type', value_name=0)
unformatted_cost['col_name'] = unformatted_cost['Cost Component'] + unformatted_cost['type']


summary = pd.concat([summary, unformatted_cost[['col_name',0]].set_index('col_name').T], axis=1)

#Portfolio Mix
portfolio_summary = portfolio[['MW','Status','Technology']].groupby(['Status','Technology']).sum().reset_index()
portfolio_summary['Description'] = portfolio_summary['Status'] + " " + portfolio_summary['Technology']
portfolio_summary = portfolio_summary.drop(columns=['Status','Technology'])
portfolio_summary = portfolio_summary.set_index('Description').transpose().reset_index(drop=True).add_prefix('MW Capacity from ')

summary = pd.concat([summary, portfolio_summary], axis=1)

#Load
summary['Customer Load GWh'] = load['zone_demand_mw'].sum(axis=0) / 1000
try:
    summary['Total Load with Storage GWh'] = (load['zone_demand_mw'].sum(axis=0) + storage_charge['ChargeMW'].sum(axis=0)) / 1000
except (KeyError, NameError):
    summary['Total Load with Storage GWh'] = summary['Customer Load GWh']

#Generation Mix
generation_summary = generation_mix.set_index('Source').transpose().reset_index(drop=True).add_prefix('GWh Generation from ')
summary = pd.concat([summary, generation_summary], axis=1)

summary = summary.transpose()
summary.columns = [f'{scenario_name}']

summary.to_csv(data_dir / 'scenario_summary.csv')